<h1 style="color: blue; text-align: center;">Multi-task Cascaded Convolutional Networks (MTCNN) for Face Detection and Facial Landmark Alignment Using Parallel</h1>

---

<h2 style="color: gray; text-align: right;">Members of Team:</h2>

| MSSV | Full Name | ID Github |
| ---- | ---- | -------- |
| 1612835 | Bùi Trọng Xuyến | btxuyenhcmus |
| 1612165 | Nguyễn Đào Vinh Hải | NDVHaiHCMUS |
| 1612859 | Nguyễn Đình Hữu | NguyenDinhHuu-HCMUS |

---
# 1.  Mô tả ứng dụng

**Input:** Một tấm ảnh RGB, Video, Video livestream.

**Output:** Một tấm ảnh RGB hoặc một video live trực tiếp được bounding các object và đánh dấu các vị trí trên khuôn mặt.

**Ý nghĩa thực tế của ứng dụng:**
- Bài toán face detection là một bài toán mang lại rất nhiều lợi ích hiện nay: auto focus đối tượng trên máy ảnh, camera giám sát, smart house, tự động phát hiện khuông mặt để tag trong các mạng xã hội, bước đầu cho bài toán face reccorgnition,...

**Ứng dụng này cần được tăng tốc**
- Khi xử lý với video khi lượng frame đưa xuống quá nhanh, kết quả trả về của mô hình không kiệp đáp ứng tạo nên độ trễ.

**Ứng dụng có thể song song hóa:**
- Song song hóa các phép tích chập trong mô hình.
- Các phép toán trên matrix.


![index](https://kpzhang93.github.io/MTCNN_face_detection_alignment/paper/examples.png)
> Nguồn ảnh: [https://kpzhang93.github.io/MTCNN_face_detection_alignment/](https://kpzhang93.github.io/MTCNN_face_detection_alignment/)

**Link to [Try step by step](try_mtcnn_step_by_step.ipynb)**

---
# 2. Cài đặt tuần tự




Training data:
- Nhóm sự dụng network được train sẵn trên bộ dữ liệu WIDER FACE bao gồm 32,203 images và label 393,703 faces.
- Bộ dữ liệu được chia thành các tập training, validation, testing với tỉ lệ 40%/10%/50%.

### Thiết kế
![index](https://kpzhang93.github.io/MTCNN_face_detection_alignment/support/index.png)
> Nguồn ảnh: [https://kpzhang93.github.io/MTCNN_face_detection_alignment/](https://kpzhang93.github.io/MTCNN_face_detection_alignment/)




#### The Three Stages of MTCNN:
Bước đầu tiên là lấy hình ảnh và thay đổi kích thước theo các tỷ lệ khác nhau để xây dựng một kim tự tháp hình ảnh, đây là đầu vào của **three-staged cascaded network**.

> **?? Tại sao chúng ta lại tạo ra một image pyramid**
> Bằng cách tăng tính đa dạng của dữ liệu, độ chính xác của mô hình MTCNN được nâng cao.

![inputimage](https://drive.google.com/file/d/14xhPMlsoKbKAcGu5jn4dkvOewfYtFY4o/view)
> Input image is resized to different scales to build an image pyramid

##### Stage 1: The Proposal Network (P-Net)

![P-net](/content/P-net.jpeg)
> P-Net (from MTCNN paper)

**P-net** là một Fully convolutional network (FCN), **P-net** được sử dụng để lấy các cửa sổ ứng viên và các vector hồi quy bao quanh. Sự khác biết giữa FCN và CNN đó là FCN không sử dụng dense layer.

Bounding box regression là công nghệ phổ biến để dự đoán khu vực của một đối tượng phân lớp, trong trường hợp này là khuôn mặt.

After that, we employ non-maximum suppression (NMS) to merge highly overlapped candidates.

**Input:** image RGB

**Output:** danh sách n-box vector (1x9)
***


##### Stage 2: The Refine Network (R-Net)

![R-net](R-net.jpeg)
> R-Net (from MTCNN paper)

Tất cả các ứng viên từ P-net sẽ được đưa vào R-net. R-net là một CNN bởi vì có một dense layer ở cuối stage. R-net giảm thêm số lượng ứng viên, thực hiện calibration các bounding box regression và NMS (non-maximum suppression) để hợp nhất các ứng viên trùng lắp.

**Input:** output của P-net

**Output:** Có phải là khuôn mặt hay không, một vector có 4 phần tử là bounding box của khuôn mặt, một vector 10 phần tử là các vị trí trong khuôn mặt.
***



##### Stage 3: The Output Network (O-Net)

![O-net](O-net.jpeg)
> R-Net (from MTCNN paper)

Giai đoạn này tương tự như R-net, nhưng Output của network này mô tả chi tiết hơn về 5 điềm của khuông mặt mắt, mũi, miệng.

**Input:** output của R-net

**Output:** Vector có 4 phần tử là bounding box của khuôn mặt, một vector 5 phần tử các vị trí của khuôn mặt


##Đánh giá: đang thực hiện
***

---
# 3. Cài đặt song song (trên GPU)

---
# 4. Cài đặt song song (trên GPU) + tối ưu hóa

---
# 5. Nhìn lại quá trình làm đồ án

---
#6. Phụ lục tham khảo

## The Three Tasks of MTCNN:
Trong tất cả các stage của Networks đều thực hiện 3 task: face/non-face classification, bounding box regression, facial landmark localization.

### 1. Face classification:
Sử dụng cross-entropy loss vì đây là bài toán phân 2 lớp, với mỗi mẫu $x_i$

$$L_i^{det} = -(y_i^{det}\log(p_i) + (1 - y_i^{det})(1-\log(p_i)))$$

Trong đó, $p_i$ là xác xuất được tạo ra bớt net chỉ ra một sample là face. $y_i^{det} \in {0, 1}$ là giá trị thật khuôn mặt hoặc không.

### 2. Bounding box regression:
Với mỗi cửa sổ ứng viên, chúng ta dự đoán các vị trí tin tưởng tạo nên một bộ offset (left top, height, width). Việc huấn luyện là bài toán regression cho nên sử dụng Euclidean loss cho mỗi sameple $x_i$:

$$L_i^{box} = \parallel\widehat{y}_i^{box} - y_i^{box}\parallel_2^2$$

Trong đó, $\widehat{y}_i^{box}$ là các vị trí dự đoán được của net, $y_i^{box}$ là tọa độ thật. Chúng ta sẽ có bốn góc, bao gồm left top, height, width và $y_i^{box} \in R^4$

### 3. Facial Landmark localization:
Tương tự như bài toán regression và chúng ta sẽ đi minimize Euclidean distance:

$$L_i^{landmark} = \parallel\widehat{y}_i^{landmark} - y_i^{landmark}\parallel_2^2$$

## Evaluate accuracy of models
#### Evaluation on face detection

Để đánh giá tính đúng đắn của mô hình, nhóm của tác giả đã so sánh với các phương pháp khác trong SOTA ()[] trên bộ dự liệu phát hiện khuôn mặt và điểm chuẩn FDDB [25], WINDER FACE [24], và dấu mốc trên khuôn mặt được chú thích trong điểm chuẩn AFLW:
    - Tập dữ liệu FDDB chứa các chú thích cho 5.171 khuôn mặt trong một tập hợp gồm 2.845 hình ảnh.
    - Bộ dữ liệu WIDER FACE bao gồm 393.703 hộp giới hạn khuôn mặt được gắn nhãn trong 32.203 hình ảnh trong đó 50% trong số đó để thử nghiệm (chia thành ba tập con theo độ khó của hình ảnh), 40% dành cho đào tạo và phần còn lại để xác thực.
    - AFLW chứa chú thích các mốc khuôn mặt cho 24.386 khuôn mặt và chúng tôi sử dụng cùng một tập hợp con thử nghiệm.

![Evaluation](./images/evaluation.png)

#### Evaluation on face alignment 

Nhóm tác giả so sánh performance với các phương pháp RCPR, TSPM, Luxand face SDK, ESR, CDM, SDM, và TCDCN. Sai số trung bình được đo bằng khoảng cách giữa các điểm mốc ước tính và độ chân thực trên mặt đất, và được chuẩn hóa đối với khoảng cách giữa hai mắt.

![Alignment](./images/alignment.png)

## Improvements with previous CNN

Nhiều CNN đã được thiết kế để nhận diện khuôn mặt. Tuy nhiên, nhóm tác giả nhận thấy rằng hiệu suất của nó có thể bị hạn chế bởi các sự kiện sau:
    - Một số bộ lọc trong các lớp tích chập thiếu tính đa dạng có thể hạn chế khả năng phân biệt của chúng.
    - So với các nhiệm vụ phân loại và phát hiện phản đối nhiều lớp khác, phát hiện khuôn mặt là một nhiệm vụ phân loại nhị phân đầy thách thức, do đó, nó có thể cần số lượng bộ lọc trên mỗi lớp ít hơn.
Để làm được điều này, nhóm tác giả giảm số lượng bộ lọc và thay đổi bộ lọc 5 × 5 thành bộ lọc 3 × 3 để giảm tính toán đồng thời tăng độ sâu để có được hiệu suất tốt hơn. Với những cải tiến này, so với kiến ​​trúc trước đó, có thể có được hiệu suất tốt hơn với thời gian chạy ít hơn.

![comparisionspeed](./images/comparisionspeed.png)

---
# 7. Tài liệu tham khảo

- [x] [PDF report](https://arxiv.org/pdf/1604.02878.pdf)
- [x] [Document Guide](https://medium.com/@iselagradilla94/multi-task-cascaded-convolutional-networks-mtcnn-for-face-detection-and-facial-landmark-alignment-7c21e8007923)
- [x] [MTCNN-Pytorch](https://github.com/TropComplique/mtcnn-pytorch)